## UTS
Muhammad Arbiansyafa Siwanto - 200411100195


## Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/prosainsdata/

/content/drive/My Drive/prosainsdata


In [3]:
#install library
!pip install sastrawi
!pip install swifter
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 5.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 KB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.5 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=78c0d7a065509d56c2fe2bdca0f4d6245b6cab65f08c0caa7297c3a78059b0ba
  Stored in directory: /root/.cache/pip/wheels/2b/5e/f2/3931524f702ffd03309e96d35ee2fbf9c61c27377511ee8d4c
Successfully built swifter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#import library
import pandas as pd
import numpy as np
from string import punctuation
import re
import nltk

In [7]:
#load data
df = pd.read_csv("tugas/dataset/dataset-baru.csv")
df.head()

,Unnamed: 0,Abstrak,Kategori
0,0,Sistem informasi akademik (SIAKAD) merupaka...,RPL
1,1,Berjalannya koneksi jaringan komputer dengan l...,RPL
2,2,Web server adalah sebuah perangkat lunak serve...,RPL
3,3,Penjadwalan kuliah di Perguruan Tinggi me...,KOMPUTASI
4,4,Seiring perkembangan teknologi yang ada diduni...,RPL


In [8]:
df.shape

(805, 3)

In [9]:
df.isnull().sum()

Unnamed: 0    0
Abstrak       0
Kategori      0
dtype: int64

In [10]:
df = df.dropna()

In [11]:
df.shape

(805, 3)

In [12]:
df.isnull().sum()

Unnamed: 0    0
Abstrak       0
Kategori      0
dtype: int64

In [13]:
df['Kategori'].value_counts()

KOMPUTASI    534
RPL          271
Name: Kategori, dtype: int64

## Preprocessing

### 1. Symbol & Punctuation Removal, case folding

In [14]:
#proses menghilangkan simbol dan emoji
def remove_text_special (text):
  text = text.replace('\\t',"").replace('\\n',"").replace('\\u',"").replace('\\',"")
  text = text.encode('ascii', 'replace').decode('ascii')
  return text.replace("http://"," ").replace("https://", " ")
df['Abstrak'] = df['Abstrak'].apply(remove_text_special)
print(df['Abstrak'])

0      Sistem  informasi  akademik  (SIAKAD) merupaka...
1      Berjalannya koneksi jaringan komputer dengan l...
2      Web server adalah sebuah perangkat lunak serve...
3      Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4      Seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    Investasi saham selama ini memiliki resiko ker...
801    Information Retrieval (IR) merupakan pengambil...
802    Klasifikasi citra merupakan proses pengelompok...
803    Identifikasi atribut pejalan kaki merupakan sa...
804    Topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object


In [ ]:
def remove_tanda_baca(text):
  text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
  return text

df['Abstrak'] = df['Abstrak'].apply(remove_tanda_baca)
df['Abstrak'].head(20)

In [ ]:
#proses menghilangkan angka
def remove_numbers (text):
  return re.sub(r"\d+", "", text)
df['Abstrak'] = df['Abstrak'].apply(remove_numbers)
df['Abstrak']

In [ ]:
#proses casefolding
def casefolding(Comment):
  Comment = Comment.lower()
  return Comment
df['Abstrak'] = df['Abstrak'].apply(casefolding)
df['Abstrak']

### 2. Word Normalization

In [ ]:
#proses tokenisasi
# from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
# def word_tokenize(text):
#   tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#   return tokenizer.tokenize(text)

df['review_token'] = df['Abstrak'].apply(lambda sentence: nltk.word_tokenize(sentence))
df['review_token']

In [ ]:
#Normalisasi kata tidak baku
normalize = pd.read_excel("tugas/dataset/Normalization Data.xlsx")

normalize_word_dict = {}

for row in normalize.iterrows():
  if row[0] not in normalize_word_dict:
    normalize_word_dict[row[0]] = row[1]

def normalized_term(comment):
  return [normalize_word_dict[term] if term in normalize_word_dict else term for term in comment]

df['comment_normalize'] = df['review_token'].apply(normalized_term)
df['comment_normalize'].head(20)

### 3. Tokenizing

### 4. Stopwords Removal

In [ ]:
#Stopword Removal
nltk.download('stopwords')
from nltk.corpus import stopwords
txt_stopwords = stopwords.words('indonesian')

def stopwords_removal(filtering) :
  filtering = [word for word in filtering if word not in txt_stopwords]
  return filtering

df['stopwords_removal'] = df['comment_normalize'].apply(stopwords_removal)
df['stopwords_removal'].head(20)

In [ ]:
#stopword removal 2
data_stopwords = pd.read_excel("tugas/dataset/list_stopwords.xlsx")
print(data_stopwords)

def stopwords_removal2(filter) :
  filter = [word for word in filter if word not in data_stopwords]
  return filter

df['stopwords_removal_final'] = df['stopwords_removal'].apply(stopwords_removal2)
df['stopwords_removal_final'].head(20)

### 5. Stemming

In [ ]:
#proses stem
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import swifter
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming (term):
  return stemmer.stem(term)

term_dict = {}
for document in df['stopwords_removal_final']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ''


In [ ]:
print(len(term_dict))
print("-----------------------------")

In [24]:
for term in term_dict:
  term_dict[term] = stemming(term)
  print(term,":",term_dict[term])

print(term_dict)
print("-----------------------------")

Output streaming akan dipotong hingga 5000 baris terakhir.
dana : dana
tunai : tunai
kredit : kredit
dealer : dealer
yamaha : yamaha
trisakti : trisakti
menjual : jual
produknya : produk
cas : cas
kontan : kontan
asuransi : asuransi
sektor : sektor
berninat : berninat
laba : laba
padapenelitian : padapenelitian
situasi : situasi
perangking : rangking
padasistem : padasistem
mysql : mysql
pengubahan : ubah
dinamis : dinamis
pengembangannya : kembang
berpikir : pikir
mendaftar : daftar
stkip : stkip
pgri : pgri
panitia : panitia
mahsiswa : mahsiswa
pemecahan : pecah
unas : unas
melepaskan : lepas
lepas : lepas
perumahan : rumah
permukiman : mukim
rtlh : rtlh
kemenpera : kemenpera
deputi : deputi
stimulan : stimulan
swadaya : swadaya
bsps : bsps
berpenghasilan : hasil
meberikan : meberikan
pramuka : pramuka
bendera : bendera
semaphore : semaphore
adobe : adobe
macromedia : macromedia
flag : flag
signaling : signaling
os : os
step : step
berurutan : rurut
praktek : praktek
mengetikkan : et

In [25]:
def get_stemming(document):
  return [term_dict[term] for term in document]

In [26]:
df['stemming'] = df['stopwords_removal_final'].swifter.apply(get_stemming)

Pandas Apply:   0%|          | 0/805 [00:00<?, ?it/s]

In [27]:
print(df['stemming'])

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [jalan, koneksi, jaring, komputer, lancar, gan...
2      [web, server, perangkat, lunak, server, fungsi...
3      [jadwal, kuliah, guru, kompleks, masalah, vari...
4      [iring, kembang, teknologi, dunia, muncul, tek...
                             ...                        
800    [investasi, saham, milik, resiko, rugi, gera, ...
801    [information, retrieval, ir, ambil, informasi,...
802    [klasifikasi, citra, proses, kelompok, piksel,...
803    [identifikasi, atribut, pejal, kaki, salah, te...
804    [topik, deteksi, objek, tarik, perhati, kemban...
Name: stemming, Length: 805, dtype: object


In [28]:
df.head(20)

,Unnamed: 0,Abstrak,Kategori,review_token,comment_normalize,stopwords_removal,stopwords_removal_final,stemming
0,0,sistem informasi akademik siakad merupaka...,RPL,"[sistem, informasi, akademik, siakad, merupaka...","[sistem, informasi, akademik, siakad, merupaka...","[sistem, informasi, akademik, siakad, sistem, ...","[sistem, informasi, akademik, siakad, sistem, ...","[sistem, informasi, akademik, siakad, sistem, ..."
1,1,berjalannya koneksi jaringan komputer dengan l...,RPL,"[berjalannya, koneksi, jaringan, komputer, den...","[berjalannya, koneksi, jaringan, komputer, den...","[berjalannya, koneksi, jaringan, komputer, lan...","[berjalannya, koneksi, jaringan, komputer, lan...","[jalan, koneksi, jaring, komputer, lancar, gan..."
2,2,web server adalah sebuah perangkat lunak serve...,RPL,"[web, server, adalah, sebuah, perangkat, lunak...","[web, server, adalah, sebuah, perangkat, lunak...","[web, server, perangkat, lunak, server, berfun...","[web, server, perangkat, lunak, server, berfun...","[web, server, perangkat, lunak, server, fungsi..."
3,3,penjadwalan kuliah di perguruan tinggi me...,KOMPUTASI,"[penjadwalan, kuliah, di, perguruan, tinggi, m...","[penjadwalan, kuliah, di, perguruan, tinggi, m...","[penjadwalan, kuliah, perguruan, kompleks, per...","[penjadwalan, kuliah, perguruan, kompleks, per...","[jadwal, kuliah, guru, kompleks, masalah, vari..."
4,4,seiring perkembangan teknologi yang ada diduni...,RPL,"[seiring, perkembangan, teknologi, yang, ada, ...","[seiring, perkembangan, teknologi, yang, ada, ...","[seiring, perkembangan, teknologi, didunia, mu...","[seiring, perkembangan, teknologi, didunia, mu...","[iring, kembang, teknologi, dunia, muncul, tek..."
5,5,gerak pekerja ada pada game yang memiliki genr...,KOMPUTASI,"[gerak, pekerja, ada, pada, game, yang, memili...","[gerak, pekerja, ada, pada, game, yang, memili...","[gerak, pekerja, game, memiliki, genre, rts, r...","[gerak, pekerja, memiliki, genre, rts, real, t...","[gerak, kerja, milik, genre, rts, real, time, ..."
6,6,perkembangan game yang semakin pesat memberik...,KOMPUTASI,"[perkembangan, game, yang, semakin, pesat, mem...","[perkembangan, game, yang, semakin, pesat, mem...","[perkembangan, game, pesat, alternative, pemin...","[perkembangan, pesat, alternative, peminatnya,...","[kembang, pesat, alternative, minat, bentuk, c..."
7,7,sistem pengenalan wajah adalah suatu sistem un...,KOMPUTASI,"[sistem, pengenalan, wajah, adalah, suatu, sis...","[sistem, pengenalan, wajah, adalah, suatu, sis...","[sistem, pengenalan, wajah, sistem, mengenali,...","[sistem, pengenalan, wajah, sistem, mengenali,...","[sistem, kenal, wajah, sistem, nali, identitas..."
8,8,teknologi mobile game beroperating system open...,KOMPUTASI,"[teknologi, mobile, game, beroperating, system...","[teknologi, mobile, game, beroperating, system...","[teknologi, mobile, game, beroperating, system...","[teknologi, mobile, beroperating, system, open...","[teknologi, mobile, beroperating, system, open..."
9,9,kantor badan kepegawaian kota bangkalan adalah...,RPL,"[kantor, badan, kepegawaian, kota, bangkalan, ...","[kantor, badan, kepegawaian, kota, bangkalan, ...","[kantor, badan, kepegawaian, kota, bangkalan, ...","[kantor, badan, kepegawaian, kota, bangkalan, ...","[kantor, badan, pegawai, kota, bangkal, instan..."


## Word Embedding Word2vec CBOW

In [29]:
import gensim
from gensim.models import Word2Vec

In [30]:
# Create CBOW model
model1 = gensim.models.Word2Vec(df['stemming'], min_count = 1,
                              vector_size = 100, window = 5)
 
# Print results
print("Cosine similarity between 'teknologi' " +
               "and 'system' - CBOW : ",
    model1.wv.similarity('teknologi', 'system'))
     
print("Cosine similarity between 'teknologi' " +
                 "and 'mobile' - CBOW : ",
      model1.wv.similarity('teknologi', 'mobile'))

Cosine similarity between 'teknologi' and 'system' - CBOW :  0.9683567
Cosine similarity between 'teknologi' and 'mobile' - CBOW :  0.9934035


In [31]:
#getting similar word
sim_words = model1.wv.most_similar('teknologi')
sim_words

[('kembang', 0.9975529909133911),
 ('pesat', 0.9961796998977661),
 ('media', 0.9946936368942261),
 ('didik', 0.9939171671867371),
 ('mobile', 0.9934035539627075),
 ('interaktif', 0.9924659729003906),
 ('satu', 0.9918490052223206),
 ('reality', 0.9913910627365112),
 ('dunia', 0.9902816414833069),
 ('bidang', 0.9902166724205017)]

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)

y_train

NameError: ignored

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components =10)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

pca.explained_variance_ratio_

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = classifier.predict(X_test)
print (" KNN Accuracy : ",
    accuracy_score(y_test,y_pred)*100)